In [ ]:
!pip install addict
!pip install -q transformers accelerate diffusers einops imageio moviepy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.6 MB/s eta 0:00:00


In [ ]:
import torch
from diffusers import DiffusionPipeline
import imageio
import numpy as np
import os
from moviepy.editor import ImageSequenceClip
from PIL import Image


  if event.key is 'enter':



In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips


In [ ]:
# !rm -rf video_frames/

In [ ]:
pipe = DiffusionPipeline.from_pretrained(
    "damo-vilab/text-to-video-ms-1.7b",
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")



The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

The TextToVideoSDPipeline has been deprecated and will not receive bug fixes or feature updates after Diffusers version 0.33.1. 


In [ ]:
def make_video(frames1):

  os.makedirs("video_frames", exist_ok=True)


  for i, frame in enumerate(frames1):
        if frame.dtype != np.uint8:
            frame = (np.clip(frame, 0, 1) * 255).astype(np.uint8)
        Image.fromarray(frame).save(f"video_frames/frame_{i:03d}.png")
        print(f"Saved frame {i}: shape={frame.shape}, dtype={frame.dtype}")


  frame_paths = sorted([f"video_frames/{f}" for f in os.listdir("video_frames") if f.endswith(".png")])
  clip = ImageSequenceClip(frame_paths, fps=8)

  looped_clip=concatenate_videoclips([clip,clip])
  looped_clip.write_videofile("/content/output.mp4", codec="libx264")

In [ ]:
import os
import numpy as np
from PIL import Image

def generate(input):

  prompt = input
  output = pipe(prompt, num_inference_steps=25, num_frames=32, height=256, width=256)
  frames = output.frames


  frames1 = frames[0]
  make_video(frames1)










In [ ]:
!pip install fastapi uvicorn python-multipart pyngrok

In [ ]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse
from pyngrok import ngrok
import shutil
import requests
import os

app = FastAPI()

from fastapi import HTTPException

@app.post("/generate/")
async def process_url(prompt: str = Form(...)):
    try:
        print(" Received prompt:", prompt)
        print(" Running pipeline...")
        generate(prompt)
        output_path = "/content/output.mp4"
        print("Pipeline completed.")

        if not os.path.exists(output_path):
            raise HTTPException(status_code=500, detail="Output video not found.")

        print(" Sending back file:", output_path)
        return FileResponse(output_path, media_type="video/mp4")

    except Exception as e:
        print(" ERROR:", str(e))
        raise HTTPException(status_code=500, detail=f"Processing failed: {str(e)}")

In [ ]:
!ngrok config add-authtoken YOUR_AUTH_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import threading
import uvicorn


# Start FastAPI server in background
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run)
thread.start()

#  Start ngrok tunnel
public_url = ngrok.connect(8000)
print(" Public URL:", public_url)

INFO:     Started server process [566]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


 Public URL: NgrokTunnel: "https://261b9b87cd95.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
torch.cuda.empty_cache()

In [ ]:
# !curl -X POST https://25611378bd97.ngrok-free.app/generate/ \
#    -F "prompt=a cute cat dancing" \
#    --output final_output.mp4

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0 Received prompt: a cute cat dancing
 Running pipeline...


  0%|          | 0/25 [00:00<?, ?it/s]

100   159    0     0  100   159      0      3  0:00:53  0:00:44  0:00:09     0Saved frame 0: shape=(256, 256, 3), dtype=uint8
Saved frame 1: shape=(256, 256, 3), dtype=uint8
Saved frame 2: shape=(256, 256, 3), dtype=uint8
Saved frame 3: shape=(256, 256, 3), dtype=uint8
Saved frame 4: shape=(256, 256, 3), dtype=uint8
Saved frame 5: shape=(256, 256, 3), dtype=uint8
Saved frame 6: shape=(256, 256, 3), dtype=uint8
Saved frame 7: shape=(256, 256, 3), dtype=uint8
Saved frame 8: shape=(256, 256, 3), dtype=uint8
Saved frame 9: shape=(256, 256, 3), dtype=uint8
Saved frame 10: shape=(256, 256, 3), dtype=uint8
Saved frame 11: shape=(256, 256, 3), dtype=uint8
Saved frame 12: shape=(256, 256, 3), dtype=uint8
Saved frame 13: shape=(256, 256, 3), dtype=uint8
Saved frame 14: shape=(256, 256, 3), dtype=uint8
Saved frame 15: shape=(256, 256, 3), dtype=uint8
Saved frame 16: shape=(256, 256, 3), dtype=uint8
Saved frame 17: shape=(256, 256, 3), dtype=uint8
Saved frame 18: shape=(256, 256, 3), dtype=uint8
S

100   159    0     0  100   159      0      3  0:00:53  0:00:46  0:00:07     0Moviepy - Done !
Moviepy - video ready /content/output.mp4
Pipeline completed.
 Sending back file: /content/output.mp4
INFO:     34.143.163.124:0 - "POST /generate/ HTTP/1.1" 200 OK
100  352k  100  351k  100   159   7504      3  0:00:53  0:00:48  0:00:05 95803


In [ ]:
# from IPython.display import HTML
# from base64 import b64encode

# # Function to display a video in the notebook
# def display_video(filename):
#     mp4 = open(filename, 'rb').read()
#     data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#     return HTML(f"""
#       <video width=640 controls autoplay loop>
#             <source src="{data_url}" type="video/mp4">
#       </video>
#       """)

# # Display the new video
# display_video("output.mp4")
